In [3]:
# install new libraries 
#!pip install sentence-transformers
#!pip install scikit-learn

In [4]:
# import libraries
import json
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import random
from datetime import datetime, timedelta
import ast

/Applications/anaconda3/envs/DSA_project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# load the JSON file
with open('all_courses.json', encoding='utf-8') as f:
    raw_data = json.load(f)

# get the actual list of courses
all_courses = raw_data['veranstaltungen']['veranstaltung']

# convert to flat DataFrame
og_df = pd.DataFrame(all_courses)

# preview result
og_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'all_courses.json'

In [ ]:
import json
import pandas as pd
with open("subset_20percent_translated_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)
subset_courses = data["events"]["event"]
mt_df = pd.json_normalize(subset_courses)
mt_df.head()

,guid,number,last_name,subtitle,district,event_type,minimum_participants,current_participants,maximum_participants,number_of_sessions,...,locations.address.city,locations.address.street,locations.address.room,locations.address.longitude,locations.address.latitude,locations.address.accessible,locations.appointments.weekday,locations.appointments.start_date,locations.appointments.start_time,locations.appointments.end_time
0,604369,FK3.840,Body Percussion – the Rhythmic Sound of the Body,None,Friedrichshain-Kreuzberg,Course,8,0,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,610723,FK4.D-ik-179-5a,Deutsch B1.1,Aufbaumodul 5,Friedrichshain-Kreuzberg,Course,12,1,20,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,656403,SZ810-29-04-01,Deutsch Integrationskurs A1.1,Basissprachkurs Modul 1,Steglitz-Zehlendorf,Course,8,6,12,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,658796,Mi404-B192S-13,Deutsch Orientierungskurs,None,Mitte,Course,8,5,16,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,683442,Sp4.224.W9,Deutsch Alphabetisierung W9,Deutsch Alphabetisierung-Integrationskurs,Spandau,Course,10,1,16,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mt_df.columns

Index(['guid', 'number', 'last_name', 'subtitle', 'district', 'event_type',
       'minimum_participants', 'current_participants', 'maximum_participants',
       'number_of_sessions', 'start_date', 'end_date', 'target_group',
       'keywords', 'text', 'category.@version', 'category.#text',
       'registration.phone', 'registration.email', 'registration.link',
       'contact_person.salutation', 'contact_person.title',
       'contact_person.last_name', 'contact_person.first_name',
       'contact_person.phone', 'contact_person.email', 'locations.address',
       'locations.appointments', 'price.amount', 'price.discount_possible',
       'price.additional', 'lecturer.salutation', 'lecturer.title',
       'lecturer.last_name', 'lecturer.first_name', 'website.type',
       'website.last_name', 'website.uri', 'lecturer',
       'merkmale.merkmal.last_name', 'merkmale.merkmal.wert',
       'locations.address.facility', 'locations.address.postal_code',
       'locations.address.city', 'loc

In [ ]:
mt_df.columns = (
    mt_df.columns
    .str.lower()
    .str.replace('.', '_', regex=False)
    .str.replace('@', '', regex=False)
    .str.replace('#', '', regex=False)
)

mt_df.head()

,guid,number,last_name,subtitle,district,event_type,minimum_participants,current_participants,maximum_participants,number_of_sessions,...,locations_address_city,locations_address_street,locations_address_room,locations_address_longitude,locations_address_latitude,locations_address_accessible,locations_appointments_weekday,locations_appointments_start_date,locations_appointments_start_time,locations_appointments_end_time
0,604369,FK3.840,Body Percussion – the Rhythmic Sound of the Body,None,Friedrichshain-Kreuzberg,Course,8,0,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,610723,FK4.D-ik-179-5a,Deutsch B1.1,Aufbaumodul 5,Friedrichshain-Kreuzberg,Course,12,1,20,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,656403,SZ810-29-04-01,Deutsch Integrationskurs A1.1,Basissprachkurs Modul 1,Steglitz-Zehlendorf,Course,8,6,12,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,658796,Mi404-B192S-13,Deutsch Orientierungskurs,None,Mitte,Course,8,5,16,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,683442,Sp4.224.W9,Deutsch Alphabetisierung W9,Deutsch Alphabetisierung-Integrationskurs,Spandau,Course,10,1,16,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mt_df['category_text'].value_counts()

category_text
3.02    266
3.01    215
4.04    177
2.07     88
4.22     87
4.06     74
5.01     73
2.11     62
5.06     57
2.09     46
4.09     43
3.05     40
4.08     34
2.08     32
5.07     25
1.06     24
2.05     23
4.24     22
4.15     21
5.03     19
2.02     19
4.01     19
5.00     18
2.13     14
4.02     14
2.10     13
4.20     11
1.01     11
5.02     10
2.04     10
4.26     10
3.03     10
1.04      8
2.00      7
2.06      7
3.06      6
4.10      5
5.08      5
5.04      5
4.14      5
1.08      4
1.02      4
1.03      4
7.00      4
1.09      4
3.00      4
4.00      3
4.19      3
1.11      3
3.04      3
7.04      2
5.05      2
4.17      1
1.00      1
2.01      1
4.16      1
1.07      1
2.12      1
Name: count, dtype: int64

In [ ]:
category_map = {
    '1.06': 'Basic Education and Literacy',
    '2.05': 'Society and Environment',
    '2.07': 'History and Politics',
    '2.08': 'Philosophy and Ethics',
    '2.09': 'Law and Justice',
    '2.11': 'Intercultural Learning',
    '3.01': 'Health and Wellbeing',
    '3.02': 'Movement and Body Awareness',
    '3.05': 'Nutrition and Cooking',
    '4.04': 'Foreign Languages',
    '4.06': 'German as a Second Language',
    '4.08': 'Integration Courses',
    '4.09': 'Literature and Writing',
    '4.15': 'Music and Voice',
    '4.22': 'Art and Creativity',
    '4.24': 'Photography and Film',
    '5.01': 'Digital Basics',
    '5.06': 'Media Skills',
    '5.07': 'Programming and Data'
}
mt_df['category_label'] = mt_df['category_text'].map(category_map)

mt_df['category_label'].value_counts()

category_label
Movement and Body Awareness     266
Health and Wellbeing            215
Foreign Languages               177
History and Politics             88
Art and Creativity               87
German as a Second Language      74
Digital Basics                   73
Intercultural Learning           62
Media Skills                     57
Law and Justice                  46
Literature and Writing           43
Nutrition and Cooking            40
Integration Courses              34
Philosophy and Ethics            32
Programming and Data             25
Basic Education and Literacy     24
Society and Environment          23
Photography and Film             22
Music and Voice                  21
Name: count, dtype: int64

In [ ]:
mt_df.to_csv("subset_20percent_translated_final.csv", index=False)

#### re-uploading the df after further translation and cleaning

In [ ]:
df = pd.read_csv("sub_20_eng.csv")
df.head(10)

,guid,course_number,course_name,course_subtitle,district,event_type,minimum_participants,current_participants,maximum_participants,number_of_sessions,...,locations_address_street,locations_address_room,locations_address_longitude,locations_address_latitude,locations_address_accessible,locations_appointments_weekday,locations_appointments_start_date,locations_appointments_start_time,locations_appointments_end_time,category_label
0,604369,FK3.840,Body Percussion – the Rhythmic Sound of the Body,NaN,Friedrichshain-Kreuzberg,Course,8,0,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,610723,FK4.D-ik-179-5a,Deutsch B1.1,Aufbaumodul 5,Friedrichshain-Kreuzberg,Course,12,1,20,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Languages
2,656403,SZ810-29-04-01,Deutsch Integrationskurs A1.1,Basissprachkurs Modul 1,Steglitz-Zehlendorf,Course,8,6,12,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Languages
3,658796,Mi404-B192S-13,Deutsch Orientierungskurs,NaN,Mitte,Course,8,5,16,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Languages
4,683442,Sp4.224.W9,Deutsch Alphabetisierung W9,Deutsch Alphabetisierung-Integrationskurs,Spandau,Course,10,1,16,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Languages
5,685434,FK3.284,Pilates Grundkurs,NaN,Friedrichshain-Kreuzberg,Course,6,0,10,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Movement and Body Awareness
6,687091,Nk-D660b1,"Deutsch - Grundstufe A2.1, Modul 5 - Zweitschr...",NaN,Neukölln,Course,6,-1,16,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Languages
7,687092,Nk-D660b2,"Deutsch - Grundstufe A2.1 - A2.2, Modul 6 - Zw...",NaN,Neukölln,Course,6,16,16,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Languages
8,687093,Nk-D660b3,"Deutsch - Grundstufe A2.2, Modul 7 - Zweitschr...",NaN,Neukölln,Course,6,16,16,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Languages
9,688705,Nk-D601b1,"Deutsch Lesen und Schreiben - Alpha III, Modul 7",NaN,Neukölln,Course,6,5,14,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foreign Languages


In [ ]:
df.isna().sum()
df.isnull().sum().sort_values(ascending=False).head(20) 

contact_person_title                 1618
lecturer_title                       1606
lecturer                             1526
locations_address_latitude           1525
locations_address_postal_code        1525
locations_address_longitude          1525
locations_address_street             1523
locations_address_city               1512
locations_address_accessible         1502
locations_appointments_weekday       1502
locations_appointments_start_date    1502
locations_address_room               1502
locations_appointments_start_time    1502
locations_address_facility           1502
locations_appointments_end_time      1502
merkmale_merkmal_wert                1490
merkmale_merkmal_last_name           1490
target_group                         1346
contact_person_phone                  831
course_subtitle                       812
dtype: int64

In [ ]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1681 entries, 0 to 1680
Data columns (total 54 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   guid                               1681 non-null   int64  
 1   course_number                      1681 non-null   object 
 2   course_name                        1681 non-null   object 
 3   course_subtitle                    869 non-null    object 
 4   district                           1681 non-null   object 
 5   event_type                         1679 non-null   object 
 6   minimum_participants               1681 non-null   int64  
 7   current_participants               1681 non-null   int64  
 8   maximum_participants               1681 non-null   int64  
 9   number_of_sessions                 1681 non-null   int64  
 10  start_date                         1681 non-null   object 
 11  end_date                           1681 non-null   objec

In [ ]:
# helper: safe parsing from string to list/dict
def safe_parse(x):
    try:
        return ast.literal_eval(x)
    except:
        return x

# helper: extract first valid field from list of dicts
def extract_first(x, key):
    x = safe_parse(x)
    if isinstance(x, list):
        for entry in x:
            if isinstance(entry, dict) and key in entry:
                return entry[key]
    return None

# helper: extract all values for a given key from list of dicts
def extract_all(x, key):
    x = safe_parse(x)
    if isinstance(x, list):
        return [entry.get(key) for entry in x if isinstance(entry, dict) and key in entry]
    return []

# flatten course description
def extract_description(x):
    x = safe_parse(x)
    if isinstance(x, list):
        for entry in x:
            if isinstance(entry, dict) and entry.get('property') == 'Description':
                return entry.get('text', '')
    return str(x)

# flatten keywords
def flatten_keywords(x):
    x = safe_parse(x)
    if isinstance(x, list):
        return ', '.join([str(i) for i in x])
    return str(x)

# flatten email from nested dict
def get_email(x):
    x = safe_parse(x)
    if isinstance(x, dict):
        return x.get('mail')
    return None

# flatten contact person full name
def get_contact_fullname(row):
    first = str(row.get('contact_person_first_name', '')).strip()
    last = str(row.get('contact_person_last_name', '')).strip()
    return f"{first} {last}".strip()

# flatten address fields (first address only)
def get_address_component(x, component):
    x = safe_parse(x)
    if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict):
        return x[0].get(component)
    return None

# price discount clean
def extract_discount_info(x):
    x = safe_parse(x)
    if isinstance(x, str):
        return x
    return ''

# apply all flattening
df['description_clean'] = df['description'].apply(extract_description)
df['keywords_clean'] = df['keywords'].apply(flatten_keywords)
df['course_weekday'] = df['locations_appointments'].apply(lambda x: extract_first(x, 'weekday'))
df['course_start_date'] = df['locations_appointments'].apply(lambda x: extract_first(x, 'start_date'))
df['course_start_time'] = df['locations_appointments'].apply(lambda x: extract_first(x, 'start_time'))
df['course_end_time'] = df['locations_appointments'].apply(lambda x: extract_first(x, 'end_time'))
df['registration_email_clean'] = df['registration_email'].apply(get_email)
df['contact_person_fullname'] = df.apply(get_contact_fullname, axis=1)

# location fields
df['address_facility'] = df['locations_address'].apply(lambda x: get_address_component(x, 'facility'))
df['address_postal_code'] = df['locations_address'].apply(lambda x: get_address_component(x, 'postal_code'))
df['address_city'] = df['locations_address'].apply(lambda x: get_address_component(x, 'city'))
df['address_street'] = df['locations_address'].apply(lambda x: get_address_component(x, 'street'))
df['address_room'] = df['locations_address'].apply(lambda x: get_address_component(x, 'room'))
df['address_longitude'] = df['locations_address'].apply(lambda x: get_address_component(x, 'longitude'))
df['address_latitude'] = df['locations_address'].apply(lambda x: get_address_component(x, 'latitude'))
df['address_accessible'] = df['locations_address'].apply(lambda x: get_address_component(x, 'accessible'))

# price info (if nested)
df['price_additional_clean'] = df['price_additional'].apply(extract_discount_info)

In [ ]:
print(df.columns.tolist())

['guid', 'course_number', 'course_name', 'course_subtitle', 'district', 'event_type', 'minimum_participants', 'current_participants', 'maximum_participants', 'number_of_sessions', 'start_date', 'end_date', 'target_group', 'keywords', 'description', 'category_version', 'category_text', 'registration_phone', 'registration_email', 'registration_link', 'contact_person_salutation', 'contact_person_title', 'contact_person_last_name', 'contact_person_first_name', 'contact_person_phone', 'contact_person_email', 'locations_address', 'locations_appointments', 'price_amount', 'price_discount_possible', 'price_additional', 'lecturer_salutation', 'lecturer_title', 'lecturer_last_name', 'lecturer_first_name', 'website_type', 'website_last_name', 'website_uri', 'lecturer', 'merkmale_merkmal_last_name', 'merkmale_merkmal_wert', 'locations_address_facility', 'locations_address_postal_code', 'locations_address_city', 'locations_address_street', 'locations_address_room', 'locations_address_longitude', 'l

In [ ]:
# running semantic search on description, keywords, and course name
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# creating a text column for search
df['search_text'] = df['course_name'].fillna('') + ' ' + df['keywords_clean'].fillna('') + ' ' + df['description_clean'].fillna('')

# loading the model and encode course data
model = SentenceTransformer('all-MiniLM-L6-v2')
course_embeddings = model.encode(df['search_text'].tolist(), show_progress_bar=True)

# defining the semantic search function
def get_semantic_matches(user_query, top_n=30):
    query_embedding = model.encode([user_query])
    similarities = cosine_similarity(query_embedding, course_embeddings)[0]
    top_indices = similarities.argsort()[-top_n:][::-1]
    matched_df = df.iloc[top_indices].copy()
    matched_df['semantic_score'] = similarities[top_indices]
    return matched_df

# run a test query
user_query = "yoga for beginners relaxation flexibility"
sem_matches = get_semantic_matches(user_query)

# first few matches
sem_matches_display = sem_matches[['guid', 'course_name', 'district', 'price_amount', 'course_weekday', 'course_start_date', 'category_label', 'description_clean', 'registration_link', 'contact_person_fullname', 'registration_email_clean', 'semantic_score']].head(10)
display(sem_matches_display)

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

,guid,course_name,district,price_amount,course_weekday,course_start_date,category_label,description_clean,registration_link,contact_person_fullname,registration_email_clean,semantic_score
284,707134,Yoga für Alle,Steglitz-Zehlendorf,59.75,Friday,2025-05-09,Health and Wellbeing,"Ein Kurs für Menschen, die Yoga ausprobieren u...",www.vhssz.de,Carolin Becker,None,0.613684
222,706761,Hatha Yoga,Steglitz-Zehlendorf,84.25,Tuesday,2025-04-29,Health and Wellbeing,Yoga bedeutet bewusste Entwicklung mit dem Zie...,www.vhssz.de,Carolin Becker,None,0.609292
1464,719216,Yoga against stress in English,Friedrichshain-Kreuzberg,61.00,Tuesday,2025-04-29,Movement and Body Awareness,Relaxation is the precondition for flow. Join ...,www.vhs-fk.de,Claudia Hiesemann,None,0.608388
345,709044,Bewegter Morgen – Gesundheit mit Yoga und Fitness,Mitte,65.00,Wednesday,2025-04-30,Health and Wellbeing,"Übungen aus dem Yoga, Pilates und der Tänzeris...",http://www.berlin.de/vhsmitte,Tanja Waldeck,None,0.607342
382,709140,Yoga für Alle,Mitte,10.00,Wednesday,2025-04-30,Health and Wellbeing,In Kooperation mit der Lebenshilfe Berlin e. V...,http://www.berlin.de/vhsmitte,Tanja Waldeck,None,0.597346
381,709137,Yoga für Alle,Mitte,10.00,Wednesday,2025-04-30,Health and Wellbeing,In Kooperation mit der Lebenshilfe Berlin e. V...,http://www.berlin.de/vhsmitte,Tanja Waldeck,None,0.597346
752,712821,Yoga und Meditation,Friedrichshain-Kreuzberg,62.00,Wednesday,2025-04-30,Movement and Body Awareness,In diesem Kurs lernen wir einige Meditationste...,www.vhs-fk.de,Claudia Hiesemann,None,0.596882
547,710316,Yoga I,Pankow,82.20,Wednesday,2025-04-30,Health and Wellbeing,Es werden Schritt für Schritt die Grundlagen d...,http://www.vhspankow.de,Angela Kalisz,None,0.594828
548,710319,Yoga I,Pankow,82.20,Wednesday,2025-04-30,Health and Wellbeing,Es werden Schritt für Schritt die Grundlagen d...,http://www.vhspankow.de,Angela Kalisz,None,0.594828
545,710309,Yoga I,Pankow,75.60,Tuesday,2025-04-29,Health and Wellbeing,Es werden Schritt für Schritt die Grundlagen d...,http://www.vhspankow.de,Angela Kalisz,None,0.594828


In [ ]:
# convert course_start_date to datetime just once
sem_matches['course_start_date'] = pd.to_datetime(sem_matches['course_start_date'], errors='coerce')

# user preferences
user_preferences = {
    'district': 'Neukölln',
    'budget_max': 100,
    'preferred_days': ['Saturday', 'Sunday'],
    'start_after': pd.to_datetime('2025-04-30'),
    'end_before': pd.to_datetime('2025-07-31')
}

# weights
weights = {
    'district': 1.0,
    'budget': 0.8,
    'weekday': 0.6,
    'start_date': 0.5
}

# rerank with proper date checks
def rerank_semantic_matches(df, prefs, weights):
    def score(row):
        s = 0

        if str(row['district']).lower() == prefs['district'].lower():
            s += weights['district']

        if pd.notnull(row['price_amount']) and row['price_amount'] <= prefs['budget_max']:
            s += weights['budget']

        if row['course_weekday'] in prefs['preferred_days']:
            s += weights['weekday']

        if pd.notnull(row['course_start_date']):
            if prefs['start_after'] <= row['course_start_date'] <= prefs['end_before']:
                s += weights['start_date']

        return s

    df['match_score'] = df.apply(score, axis=1)
    return df.sort_values(by='match_score', ascending=False)

# apply it
ranked_df = rerank_semantic_matches(sem_matches, user_preferences, weights)

# display final ranked result
display(ranked_df[['guid', 'course_name', 'district', 'price_amount', 'course_weekday', 'course_start_date', 'category_label', 'match_score']].head(10))

,guid,course_name,district,price_amount,course_weekday,course_start_date,category_label,match_score
319,708972,Yin Yoga und die innere Welt,Mitte,62.90,Saturday,2025-06-21,Health and Wellbeing,1.9
284,707134,Yoga für Alle,Steglitz-Zehlendorf,59.75,Friday,2025-05-09,Health and Wellbeing,1.3
548,710319,Yoga I,Pankow,82.20,Wednesday,2025-04-30,Health and Wellbeing,1.3
554,710335,Yin Yoga,Pankow,69.00,Friday,2025-05-09,Movement and Body Awareness,1.3
553,710331,Yoga II,Pankow,69.00,Tuesday,2025-05-06,Health and Wellbeing,1.3
552,710329,Yoga II,Pankow,62.40,Thursday,2025-05-15,Health and Wellbeing,1.3
283,707133,Hatha Yoga - achtsamkeitsbasiert,Steglitz-Zehlendorf,70.25,Friday,2025-05-09,Health and Wellbeing,1.3
551,710327,Yoga II,Pankow,82.20,Wednesday,2025-04-30,Health and Wellbeing,1.3
951,714617,Yoga für Männer (Einsteiger),Friedrichshain-Kreuzberg,53.40,Thursday,2025-05-15,Health and Wellbeing,1.3
547,710316,Yoga I,Pankow,82.20,Wednesday,2025-04-30,Health and Wellbeing,1.3
